Calculate ensemble of sea level projections

In [1]:
import numpy as np
import xarray as xr

import sys
sys.path.append('../src')
import utils as ut

In [2]:
#year0 = 1851
year0 = 1951

option = ''
#option = '_fit'


year1 = year0+149

In [3]:
ds = xr.open_dataset(f'../data/alldata{option}_{year0}_{year1}.nc')

ds = ds.isel(exp=slice(0,5))

#Extract few models for quick calcs
#ds = ds.isel(esm=slice(0,2),ism=slice(0,2))

In [4]:
for b,bmp in enumerate(ut.bmps):
    slr_nf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    slr_wf = np.zeros((len(ds.ssp),len(ds.esm),len(ds.ism),len(ds.time)))
    for s,ss in enumerate(ds.ssp.values):
        for e,es in enumerate(ds.esm.values):
            for i,ii in enumerate(ds.ism.values):
                TMP,IML,SLR = ut.iterate(ds,ism=ii,esm=es,ssp=ss,bmp=bmp)
                slr_nf[s,e,i,:] = np.sum(SLR[0,:,:],axis=1)
                slr_wf[s,e,i,:] = np.sum(SLR[-1,:,:],axis=1)
                print(f'{bmp:4s}',ss,f'{es:14s}',f'{ii:10s}',f'{slr_nf[s,e,i,-1]: .3f}',f'{slr_wf[s,e,i,-1]: .3f}')       
    #Save
    slr_nf2 = xr.DataArray(slr_nf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise without feedback'})
    slr_wf2 = xr.DataArray(slr_wf,dims=('ssp','esm','ism','time'),coords={'ssp':ds.ssp,'esm':ds.esm,'ism':ds.ism,'time':ds.time},attrs={'unit':'m','long_name':'Cumulative sea level rise with feedback'})

    ds2 = xr.Dataset({'slr_nf':slr_nf2,'slr_wf':slr_wf2})
    ds2.to_netcdf(f'../data/ensemble{option}_{bmp}_{year0}_{year1}.nc',mode='w')
    ds2.close()
    print(f'Saved {bmp}')
ds.close()

lin  126 ACCESS-CM2     CISM_NCA    0.042  0.115
lin  126 ACCESS-CM2     FETI_ULB    0.061  0.227
lin  126 ACCESS-CM2     GRIS_LSC    0.014  0.036
lin  126 ACCESS-CM2     IMAU_UU     0.147  0.443
lin  126 ACCESS-CM2     ISSM_UCI    0.126  0.263
lin  126 ACCESS-CM2     MALI_DOE    0.066  0.182
lin  126 ACCESS-CM2     PISM_DMI    0.122  0.659
lin  126 ACCESS-CM2     PISM_PIK    0.059  0.178
lin  126 ACCESS-CM2     SICO_ILTS   0.132  0.474
lin  126 CAMS-CSM1-0    CISM_NCA    0.042  0.128
lin  126 CAMS-CSM1-0    FETI_ULB    0.058  0.261
lin  126 CAMS-CSM1-0    GRIS_LSC    0.025  0.063
lin  126 CAMS-CSM1-0    IMAU_UU     0.137  0.456
lin  126 CAMS-CSM1-0    ISSM_UCI    0.118  0.278
lin  126 CAMS-CSM1-0    MALI_DOE    0.066  0.210
lin  126 CAMS-CSM1-0    PISM_DMI    0.107  0.921
lin  126 CAMS-CSM1-0    PISM_PIK    0.064  0.180
lin  126 CAMS-CSM1-0    SICO_ILTS   0.136  0.562
lin  126 CAS-ESM2-0     CISM_NCA    0.116  0.505


KeyboardInterrupt: 